In [1]:
#purpose is lowering the memory


#https://github.com/jonas-pettersson/fast-ai/blob/master/Exploration%20and%20Prediction%20for%20Structured%20Data.ipynb
#https://forums.fast.ai/t/unofficial-lesson-3-classnotes/7567
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

/home/hafiz/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
import feather
df_raw = feather.read_dataframe('/tmp/raw')
print('import complete')

In [ ]:
#used to reduce memory

types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

#objects are create general purpose python data type

#onpromotion iis boolean having missing value

In [ ]:
%%time
df_all = pd.read_csv(f'{PATH}train_basic.csv', parse_dates=['date'], dtype=types, 
                     infer_datetime_format=True)#, skiprows=range(1,100000000))

In [ ]:


#onpromotion iis boolean having missing value so filled the missing
#value = false EDA says missing =false is not approperiate
df_all.onpromotion.fillna(False, inplace=True)
#above you can see the datatype is object for onpromotion by default 
#string so we replace it with boolean
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
#finally changed the data type
df_all.onpromotion = df_all.onpromotion.astype(bool)

#by doing this 123 million rows take 2.5 gigabytes of datatype


os.makedirs('tmp', exist_ok=True)

#look you can save it in feather format under 5 seconds
%time df_all.to_feather('tmp/raw_groceries')

In [ ]:
#see the time is 23.1s

%time df_all.describe(include='all')

In [ ]:
#first important thing is look at dates 

#date is important among the other stuff

#anything in the world changes then you need to know
#how your predictive accuracy changes as well

#make sure your date dont overlap
#here we have date 2013 to 2017

#

In [ ]:
#now look at test dataset

In [ ]:
df_test = pd.read_csv(f'{PATH}test.csv', parse_dates = ['date'],
                      dtype=types, infer_datetime_format=True)
df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_test.onpromotion.map({'False': False, 
                                               'True': True})
df_test.onpromotion = df_test.onpromotion.astype(bool)
df_test.describe(include='all')

In [ ]:
#here you can see in test data dates are 1 day later
#to summarize the discussion you can see in training set
#you have 3 year of data and in testing set you are predicting
#next 3 weeks of data

# what if you have small dataset then you should not do random 
#sampling just sample the most recent

In [ ]:
#Ques

df_all.tail()

In [ ]:

# In competetion there is some negative sales
# which means return they consider that we should consider 
#them zero
#so the sales falls between zero and no perticular maximum9none

#clip that and take log +1 rms

df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [ ]:
%time add_datepart(df_all, 'date')

In [ ]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

In [ ]:
#all of the dataset is already numeric

# train_cats(raw_train)
# apply_cats(raw_valid, raw_train)

In [ ]:
%%time
trn, y, nas = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales')

In [ ]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())
`
def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
          m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
set_rf_samples(1000000)

In [ ]:
%time x = np.array(trn, dtype=np.float32)

In [ ]:
# set n_jobs=-1 to use all CPU cores available
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=-1) 
%time m.fit(x, y)
#%prun m.fit(x, y) to check which line of code take how much time
#why did not do oob score then it will take other million
#rows to calculate which means forever


In [ ]:
#average cell sale in month of august,

#how about performance of new model with new column on x-axis and
#y-axis previous model

#tackling this problem with creating new columns average sale, inference
#column data

In [ ]:
#rossmann store sales Blog kaggle to see that

In [ ]:
#significance of validation set

#47:28 kaggle laderboard score and vidation test score identical 
#see plot